> This notebook is replicated from Kavya's GbmMINERRegulonValidation.ipynb for regulon validation of TCGA and additional datasets

# Load modules

In [1]:
from __future__ import division
import multiprocessing
import time,datetime
import sys
import os
from sklearn.decomposition import PCA
import numpy as np
import miner_py3 as miner
import imp
imp.reload(miner)
import pandas as pd
import warnings
from tqdm.notebook import tqdm, trange
warnings.filterwarnings("ignore")
print("Last updated:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Last updated: 2022-04-26 14:54:08


# Load data

In [2]:
# define data and resulting model directory
dataDirectory = '/Volumes/omics4tb2/SYGNAL/GBM-Serdar/data/'
resultsDirectory='/Volumes/omics4tb2/SYGNAL/GBM-Serdar/MINER_All_ST030222/'
os.chdir(resultsDirectory)

# load expression data
expressionDataTCGA=pd.read_csv(os.path.join(dataDirectory,"GbmMicroRNAMergedWithIDsZScored.csv"),header=0,index_col=0,sep=",")
#expressionDataFrench=pd.read_csv(os.path.join(dataDirectory,"French_MinerInput.csv"),header=0,index_col=0,sep=",")
#expressionDataRembrandt=pd.read_csv(os.path.join(dataDirectory,"Rembrandt_MinerInput.csv"),header=0,index_col=0,sep=",")

# Load regulon info
regulonDf = pd.read_csv("regulonDf.csv",header=0,sep=",", index_col=0)
coherentMemberDf = pd.read_csv("coherentMembers.csv",header=0,index_col = 0, sep=",")
regulonids = list(regulonDf['Regulon_ID'].drop_duplicates())
print("Loaded\n expression data:", expressionDataTCGA.shape,"\n",
      "Regulons:", len(regulonids))

Loaded
 expression data: (9728, 548) 
 Regulons: 4341


# Regulon validation functions

In [3]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def regulonValidation(expressionData,regulonDf,regulonids,coherentMemberDf):
    validationlist = list()
    for i in tqdm(regulonids):
        time.sleep(0.01)
        
        regulonidgenes = regulonDf[regulonDf['Regulon_ID']==i]['Gene']
        regulonidgenes = list(set(regulonidgenes))

        coherentmembers = [k for k in coherentMemberDf.columns if coherentMemberDf.loc[i,k] == 1]

        if(len(coherentmembers) != 0):
            othergenes = regulonDf[regulonDf['Regulon_ID']!=i]['Gene']
            othergenes = list(set(othergenes))

            # expression of genes in the regulon
            subsetexp = expressionData.loc[regulonidgenes, coherentmembers]
            subsetexp.replace("", np.nan, inplace=True)
            subsetexp = subsetexp.dropna()

            # expression of the genes not in regulon
            subsetexpnon = expressionData.loc[othergenes, coherentmembers]
            subsetexpnon.replace("", np.nan, inplace=True)
            subsetexpnon = subsetexpnon.dropna()

            # PCA included genes
            pca1 = PCA(1, random_state=12)
            principalComponents1 = pca1.fit_transform(subsetexp.T)
            var_main = pca1.explained_variance_ratio_

            # PCA excluded genes
            pca2 = PCA(1, random_state=12)
            principalComponents2 = pca2.fit_transform(subsetexpnon.T)
            var_non = pca2.explained_variance_ratio_

            # expression and PCA of noncohorent members
            noncoherentmembers = [k for k in coherentMemberDf.columns if coherentMemberDf.loc[i,k] == 0]
            subsetexpnonmem = expressionData.loc[regulonidgenes, noncoherentmembers]
            subsetexpnonmem.replace("", np.nan, inplace=True)
            subsetexpnonmem = subsetexpnonmem.dropna()
            pca4 = PCA(1, random_state=12)
            principalComponents4 = pca4.fit_transform(subsetexpnonmem.T)
            var_nonmem = pca4.explained_variance_ratio_

            good_calls = []
            good_callsnonmem = []
            all_calls = []
            all_callsnonmem = []
            for j in range(500):
                expressionDataTrim = expressionData.loc[:, coherentmembers]
                random_subset = expressionDataTrim.sample(n=len(regulonidgenes))
                random_subset.replace("", np.nan, inplace=True)
                random_subset = random_subset.dropna()
                pca3 = PCA(1, random_state=12)
                principalComponents3 = pca3.fit_transform(random_subset.T)
                var_rand = pca3.explained_variance_ratio_
                all_calls.append(var_rand)
                if var_rand >= var_main:
                    good_calls.append(var_rand)
                j+=1

                expressionDataTrim = expressionData.loc[regulonidgenes, :]
                random_subset = expressionDataTrim.sample(len(coherentmembers), axis=1)
                random_subset.replace("", np.nan, inplace=True)
                random_subset = random_subset.dropna()
                pca5 = PCA(1, random_state=12)
                principalComponents5 = pca5.fit_transform(random_subset.T)
                var_rand = pca5.explained_variance_ratio_
                all_callsnonmem.append(var_rand)
                if var_rand >= var_main:
                    good_callsnonmem.append(var_rand)
                j+=1

            random_permute_pval = float((len(good_calls)+1)/501)
            random_permute_var_avg = sum(all_calls)/len(all_calls)
            diff_non = var_main - var_non
            diff_rand = var_main - random_permute_var_avg

            random_permute_pval_nonmem = float((len(good_callsnonmem)+1)/501)
            random_permute_var_nonmem_avg = sum(all_callsnonmem)/len(all_callsnonmem)
            diff_nonmem = var_main - var_nonmem
            diff_randnonmem = var_main - random_permute_var_nonmem_avg
            validationlist.append([int(i),float(var_main),float(var_non),float(random_permute_var_avg), len(good_calls), random_permute_pval,float(diff_non),float(diff_rand), float(var_nonmem),float(random_permute_var_nonmem_avg), len(good_callsnonmem), random_permute_pval_nonmem,float(diff_nonmem),float(diff_randnonmem)])
    df = pd.DataFrame(validationlist) 
    dfnodup = df.drop_duplicates()
    dfnodup.columns = ['Regulon_ID', 'PCA_1Var', 'PCA_1VarNonRegulonGeneSet', 'PCA_1RandomVarAvg', 'NumGoodCalls','PCA_1RandomPVal', 'Diff_PCA_1VarRegulonVsNonRegulon', 'Diff_PCA_1VarRegulonVsAvgRand', 'PCA_1VarNonMemberGeneSet', 'PCA_1RandomVarAvgNonMember', 'NumGoodCallsNonMember','PCA_1RandomPValNonMember', 'Diff_PCA_1VarRegulonVsNonMember', 'Diff_PCA_1VarRegulonVsAvgRandNonMember']
    return dfnodup

# Run Regulon validation for TCGA

In [4]:
tcga_validation = regulonValidation(expressionDataTCGA,regulonDf,regulonids,coherentMemberDf)

  0%|          | 0/4341 [00:00<?, ?it/s]

# Write to a file

In [5]:
tcga_validation.to_csv("TCGA_Regulon_Variance_Test.csv") 

# Run Regulon validation for French

In [4]:
expressionDataFrench=pd.read_csv(os.path.join(dataDirectory,"French_MinerInput.csv"),header=0,index_col=0,sep=",")

# Load regulon info
regulonDf = pd.read_csv("regulonDf.csv",header=0,sep=",", index_col=0)

coherentMemberDfFrench = pd.read_csv("coherentMembersFrench.csv",header=0,index_col = 0, sep=",")
regulonids = list(regulonDf['Regulon_ID'].drop_duplicates())
print("Loaded\n expression data:", expressionDataFrench.shape,"\n",
      "Regulons:", len(regulonids))

Loaded
 expression data: (11643, 252) 
 Regulons: 4341


In [5]:
i=0
regulonidgenes = regulonDf[regulonDf['Regulon_ID']==i]['Gene']
regulonidgenes
regulonidgenes = list(set(regulonidgenes))
coherentmembers = [k for k in coherentMemberDfFrench.columns if coherentMemberDfFrench.loc[i,k] == 1]
coherentmembers


subsetexp = expressionDataFrench.loc[regulonidgenes, coherentmembers]
subsetexp.replace("", np.nan, inplace=True)
subsetexp = subsetexp.dropna()


KeyError: "['ENSG00000129450', 'ENSG00000103522', 'ENSG00000136305', 'ENSG00000282076', 'ENSG00000119535'] not in index"

In [24]:
french_validation = regulonValidation(expressionDataFrench,regulonDf,regulonids,coherentMemberDfFrench)

  0%|          | 0/4341 [00:00<?, ?it/s]

KeyError: "['ENSG00000119535', 'ENSG00000129450', 'ENSG00000282076', 'ENSG00000136305', 'ENSG00000103522'] not in index"